In [0]:
# # 02 - Ingeniería de Features

# COMMAND ----------

from pyspark.sql.functions import avg

# Leer datos limpios desde tabla
df = spark.table("datalottery.lotterybets.lottery_bets_dirty_cleaned")

# Promedio de monto por usuario
avg_stake = df.groupBy("user_id").agg(avg("stake_amount").alias("avg_stake_amount"))

# Selección de features
features_df = df.select(
    "user_id",
    "bets_last_7d",
    "win_rate_last_30d",
    "ip_risk",
    "geo_risk",
    "num_picks",
    "suspicious"
).dropDuplicates(["user_id"])

# Unión con el promedio
features_df = features_df.join(avg_stake, on="user_id", how="left")

# Guardar como tabla Delta
features_df.write.format("delta").mode("overwrite").saveAsTable("datalottery.lotterybets.lottery_bets_dirty_features")